In [ ]:
#####################################################################################################
##Copyright    O., Hamel et S. Lamari for The C00L07UN100120180002 Project.##########################
#####################################################################################################

#this file contains the code relating to the training of the Bi LSTM paraphrase generation model (Arabic).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import the linguistic resources

%tensorflow_version 1.x
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


downloaded = drive.CreateFile({'id':'1Dn7Dm5qQhGB2zEG38Vo3OgN2t6_5KUFs'}) 
downloaded.GetContentFile('W2V.npy')

downloaded = drive.CreateFile({'id':'1We-uATZvQ5Qm2LM0f8z6yfHzS6i6FmV6'}) 
downloaded.GetContentFile('W2I.npy')

downloaded = drive.CreateFile({'id':'1rlYGeW2HQK4umbRD0c-uzCuEiKUmbVGq'}) 
downloaded.GetContentFile('I2W.npy')

downloaded = drive.CreateFile({'id':'1wWjjQe4QwZLDKvq_iKsQVIFYzPxqfQrH'}) 
downloaded.GetContentFile('Classeur2.csv')

import csv
import numpy as np

# read and clean the dataset

x = list()
y = list()

file=open("Classeur2.csv","r",encoding="utf8")
test=csv.reader(file)
i = -1
for row in test:
    sentence = ''.join(map(str, row))  #convertir la list en string
    wordList = sentence.split()        # convertir la phrase en une liste de mots
    
    i = i+1
    if i%2==0:
        x.append(wordList)
    else:
        y.append(wordList)

word_to_vec,word_to_int,int_to_word,data = {},{},{},{}

data = np.load('W2V.npy', allow_pickle =True)
word_to_vec.update(data.item()) 

data = np.load('W2I.npy', allow_pickle =True) 
word_to_int.update(data.item()) 

data =  np.load('I2W.npy', allow_pickle =True)
int_to_word.update(data.item())

c = np.zeros(300)
word_to_vec[' ']=c
c=[]


word_to_int = {k:v+1 for k,v in word_to_int.items()}
word_to_int[' '] = 0

int_to_word = {v:k for k,v in word_to_int.items()}


int_to_vec={}

int_to_vec = {i:word_to_vec[int_to_word[i]] for i in int_to_word.keys()}

print('la taille de x : ',len(x))
print('la taille de y : ',len(y))

downloaded = []

data = [(e1,e2) for e1,e2 in zip(x,y) if len(e1)<=30 and len(e2)<=30] 

print(len(data))

x_data = [] # list of lists
y_data = [] # list of lists

for (e1,e2) in data : 
  e1 = e1 + [' ']* (30 - len (e1))
  e2 = e2 + [' ']* (30 - len (e2))

  x_data.append(e1)
  
  y_data.append(e2)
  

print(len(x_data))
print(len(y_data))

In [ ]:
# import packages

import re
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential,Model
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Bidirectional, Input, LSTM
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy, mean_squared_error, mean_absolute_error, logcosh
from keras.layers.normalization import BatchNormalization
from keras import regularizers

import math
import random
import sys
import matplotlib.pyplot as plt
from keras.utils import plot_model

num_train_patters = len(x_data)
print('Total patterns:\t' + str(num_train_patters))

Using TensorFlow backend.


Total patterns:	81102


In [ ]:
#define parameters

rnn_size = 256 # size of RNN
learning_rate = 0.0001 #learning rate
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf

print('Building LSTM model...')

#create the model

model = Sequential()
model.add(Bidirectional(LSTM(rnn_size, activation="relu",return_sequences = True),input_shape=(30, 300)))
model.add(Dropout(0.3))
model.add(Dense(len(int_to_vec),activation="softmax"))
optimizer = Adam(lr=learning_rate)
loss_function_used = 'categorical_crossentropy'
callbacks=[EarlyStopping(patience=2, monitor='val_acc')]
model.compile(optimizer='Adam',loss= loss_function_used,metrics=['acc'])
print('LSTM model built.')

Building LSTM model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
LSTM model built.


In [ ]:
# data partition part

import numpy as np
import keras
import csv
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow import keras
import numpy as np
from gensim.models import KeyedVectors
from gensim.models import Word2Vec


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1, n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        global x_data
        global y_data
        self.x_data,self.y_data= x_data, y_data
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        global word_to_vec
        global word_to_int

        X = np.zeros((self.batch_size, 30,300),'float64')
        Y = np.zeros((self.batch_size, 30,len(word_to_vec)),'float64')
        

        #for i, ID in enumerate(list_IDs_temp):
        for i,ID in enumerate(list_IDs_temp) :
            sentence = self.x_data[ID]
            paraphrase = self.y_data[ID]
            pos = 0
            for j in sentence:
                X[i, pos,] = word_to_vec[j]
                pos = pos+1
            pos = 0
            for j in paraphrase:
                Y[i, pos,word_to_int[j]] = 1
                pos = pos+1

                
        return X, Y


In [ ]:
#Train part

import keras 

params = {'dim': (32,32,32),
          'batch_size': 32,
          'n_channels': 1,
          'n_classes': 6,
          'shuffle': True
          }

# Datasets

trainingSet = [i for i in range(0,64881)]
validationSet = trainingSet[48661:64881]
trainingSet = trainingSet[0:48661]


partition = {}
partition['train'] = trainingSet
partition['validation'] = validationSet


# Generators
training_generator = DataGenerator(partition['train'], y_data, **params)
validation_generator = DataGenerator(partition['validation'], y_data, **params)

"""
recuperate the model in order to continue training
model = keras.models.load_model('/content/drive/My Drive/Arabic DataSet 1sens 32/bestmodel.h5')

""""

print(model.summary())

acc  = float('-inf')

print(model.summary())

for i in range (500):

  history = model.fit_generator(generator=training_generator,validation_data=validation_generator,epochs = 1,use_multiprocessing=True,workers=1)
  
  #recuperation of loss & acc

  curr_loss = history.history['loss'][0]
  curr_acc = history.history['acc'][0]
  curr_val_acc = history.history['val_acc'][0] 
  curr_val_loss = history.history['val_loss'][0]

  loss_file=open("/content/drive/My Drive/Arabic DataSet 1sens 32/lossOneHotBiLSTM.txt", "a+")
  acc_file=open("/content/drive/My Drive/Arabic DataSet 1sens 32/accOneHotBiLSTM.txt", "a+")
                                                                            
  Valloss_file=open("/content/drive/My Drive/Arabic DataSet 1sens 32/VallossOneHotBiLSTM.txt", "a+")
  Valacc_file=open("/content/drive/My Drive/Arabic DataSet 1sens 32/ValaccOneHotBiLSTM.txt", "a+")
                                                                 
  Valloss_file.write('\n'+str(curr_val_loss))
  Valacc_file.write('\n'+str(curr_val_acc))

  loss_file.write('\n'+str(curr_loss))
  acc_file.write('\n'+str(curr_acc))
  
  loss_file.close()
  acc_file.close()
  Valacc_file.close()
  Valloss_file.close()
  
  #saving model

  if curr_acc > acc :
    model.save('/content/drive/My Drive/Arabic DataSet 1sens 32/bestmodel.h5')
    acc = curr_acc
  else:
    model.save('/content/drive/My Drive/Arabic DataSet 1sens 32/model.h5')
  